In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Kaggle Workshops Starter Notebook [Python version]
## FinTech: Predicting defaults on credit loans

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?name=FinTech%3A%20Credit%20Risk&download_url=https%3A%2F%2Fgist.githubusercontent.com%2Foliviervg1%2Fa8f94d673be2aa7c6cffe384a582c9cc%2Fraw%2Fcredit_risk_bqml_python.ipynb&url=https%3A%2F%2Fgist.github.com%2Foliviervg1%2Fa8f94d673be2aa7c6cffe384a582c9cc">
      <img src="https://cloud.google.com/images/products/ai/ai-solutions-icon.svg" alt="Vertex AI Notebooks">Run on Vertex AI Notebooks</a>
  </td>
</table>

- [Setup BigQuery in Jupyter](#setup)
- [BigQuery ML basics](#basics)
  - [Select BigQuery data from Python](#select)
  - [Train a BigQuery ML model](#train)
  - [Check models currently being trained on BigQuery](#currentlytraining)
  - [Check if a model has finished training](#finishedtraining)
  - [Evaluate a BigQuery ML model](#evaluate)
  - [Compare multiple models](#compare)
  - [ML.FEATURE_INFO to check input feature information](#featinfo)
  - [Make predictions using BigQuery ML](#predictions)
  - [Export predictions as a CSV file](#exportcsv)
- [Feature engineering](#feateng)
  - [EXTRACT: Extract dates and time](#extractdates)
  - [REGEXP_EXTRACT_ALL: Extract words from a STRING column](#extractwords)
  - [ML.QUANTILE_BUCKETIZE: Bucketize your numeric feature into bins](#bucketize)
  - [ML.POLYNOMIAL_EXPAND: Combine numeric features](#polynomialexpand)
  - [ML.FEATURE_CROSS: Combine categorical features](#featurecross)
- [Feature selection](#featselection)
  - [ML.FEATURE_IMPORTANCE for Boosted Tree models](#featimportance)

<a id="setup"></a>

### Setup BigQuery in Jupyter 

#### Set parameters

In [ ]:
#Check current project
GCP_PROJECTS = !gcloud config get-value project

In [ ]:
PROJECT_ID = GCP_PROJECTS[0]
DATASET_ID = "kaggle"
LOCATION = "US"

#### Create dataset if it does not yet exist

In [ ]:
!bq mk --location=$LOCATION $DATASET_ID

#### Required libraries

In [ ]:
!pip install google-cloud-bigquery google-cloud-bigquery-storage

In [ ]:
from google.cloud import bigquery
from google.api_core.exceptions import BadRequest
import time
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

# Construct a BigQuery client object.
client = bigquery.Client()

def bq_query(sql, async_flag=False):
    """
    If `async_flag` set to False (default), 
        returns the query results for `sql` as a Pandas DataFrame, 
    Else, submits query asynchronously and returns nothing.
    """
    
    # Try dry run before executing query to catch any errors
    try:
        job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
        dry_run_job = client.query(sql, job_config=job_config)
    except BadRequest as err:
        print(err)
        return
        
    job_config = bigquery.QueryJobConfig()
    df = client.query(sql, job_config=job_config)
    
    if async_flag:
        return f"Query sent asynchronously for job id: {df.job_id}"
    else:
        df = df.result() #wait for query to finish running
        return df.to_dataframe()

<a id="basics"></a>

## BigQuery ML basics

<a id="select"></a>

#### Select BigQuery data from Python

In [ ]:
sql_select = """
SELECT 
  *
FROM
  kaggle-hackathon.credit.train
LIMIT 10
"""

df = bq_query(sql = sql_select)

df.head()

<a id="train"></a>

#### Train a BigQuery ML model

In [ ]:
MODEL_NAME = "is_loan_bad_classifier"

sql_logreg_sample = f"""
CREATE OR REPLACE MODEL
 `{DATASET_ID}.{MODEL_NAME}`
  OPTIONS(
    MODEL_TYPE = 'LOGISTIC_REG',
    INPUT_LABEL_COLS = ['is_loan_bad'],
    AUTO_CLASS_WEIGHTS = TRUE,
    ENABLE_GLOBAL_EXPLAIN = TRUE
  )
AS
SELECT
 loan_amnt,
 term,
 annual_inc,
 is_loan_bad
FROM
 `kaggle-hackathon.credit.train`
"""

# send asynchronously
bq_query(sql = sql_logreg_sample, async_flag=True)

Other model types for classification:
 - [`logistic_reg`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create)
 - [`boosted_tree_classifier`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-boosted-tree)
 - [`dnn_classifier`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-dnn-models)
 - [`automl_classifier`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-automl)

<a id="currentlytraining"></a>

#### Check models currently being trained on BigQuery

This will check if the models in your project are still undergoing training.

In [ ]:
sql_currentlyrunningqueries = f"""
SELECT
    job_id,
    creation_time,
    query,
    REGEXP_EXTRACT(query, r'{DATASET_ID}\.\w+') AS modelname
FROM `region-us`.INFORMATION_SCHEMA.JOBS_BY_USER
WHERE 
    REGEXP_CONTAINS(LOWER(query), 'create (or replace ){{0,1}}model')
    AND NOT REGEXP_CONTAINS(LOWER(query), 'information_schema')
    AND state != "DONE"
"""

print(sql_currentlyrunningqueries)

# send asynchronously
models_still_training = bq_query(sql_currentlyrunningqueries)

print(f"{len(models_still_training)} models still currently training:")
models_still_training

<a id="finishedtraining"></a>

#### Check which models have finished training

In [ ]:
sql_finished_queries = f"""
SELECT
    job_id,
    creation_time,
    query,
    REGEXP_EXTRACT(query, r'{DATASET_ID}\.\w+') AS modelname
FROM `region-us`.INFORMATION_SCHEMA.JOBS_BY_USER
WHERE 
    REGEXP_CONTAINS(LOWER(query), 'create (or replace ){{0,1}}model')
    AND NOT REGEXP_CONTAINS(LOWER(query), 'information_schema')
    AND state = "DONE"
ORDER BY creation_time DESC
"""

print(sql_finished_queries)

finished_models = bq_query(sql_finished_queries)
finished_models.head()

<a id="evaluate"></a>

#### Evaluate a BigQuery ML model

In [ ]:
MODEL_NAME = "is_loan_bad_classifier"

sql_logreg_sample = f"""
SELECT 
  *
FROM
  ML.EVALUATE(
      MODEL {DATASET_ID}.{MODEL_NAME})
"""

print(sql_logreg_sample)

bq_query(sql = sql_logreg_sample)

<a id="compare"></a>

### Compare multiple models

In [ ]:
MODEL_LIST = ["is_loan_bad_classifier", 
              "is_loan_bad_classifier", 
              "is_loan_bad_classifier"] #change or add models here

sql_modeleval_list = [f"SELECT \n  '{modelname}' as modelname, \n  * \nFROM \n  ML.EVALUATE(MODEL {DATASET_ID}.{modelname})"
                      for modelname 
                      in MODEL_LIST]

sql_compare_evaluation = "\n\nUNION ALL\n\n".join(sql_modeleval_list)

print(sql_compare_evaluation)

bq_query(sql_compare_evaluation)

<a id="featinfo"></a>

### ML.FEATURE_INFO to check input feature information

The ML.FEATURE_INFO function allows you to see information about the input features used to train a model. [ML.FEATURE_INFO Documentation](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-feature)

In [ ]:
MODEL_NAME = "is_loan_bad_classifier"

sql_featureinfo = f"""
SELECT
    *
FROM
    ML.FEATURE_INFO(MODEL {DATASET_ID}.{MODEL_NAME})
"""

bq_query(sql_featureinfo)

<a id="predictions"></a>

### Make predictions using BigQuery ML

In [ ]:
MODEL_NAME = "is_loan_bad_classifier"

sql_predict = f"""
SELECT
  id, 
  prob as is_loan_bad
FROM
  ML.PREDICT(MODEL {DATASET_ID}.{MODEL_NAME},
    (
    SELECT 
      *
    FROM
    `kaggle-hackathon.credit.test`
  )),
UNNEST(predicted_is_loan_bad_probs)
WHERE label = 1
"""

print(sql_predict)

submission = bq_query(sql_predict)
submission.head()

<a id="exportcsv"></a>

### Export predictions for Kaggle as a CSV file

In [ ]:
MODEL_NAME = "is_loan_bad_classifier"

#save as CSV
submission.to_csv(f"submission_{MODEL_NAME}.csv", index=False)

The predictions are now saved to a CSV file. To download the file to your computer, locate your file in the left-menu in JupyterLab, right-click on it and click "Download". You can now make a submission to the Kaggle competition with the file.

<hr><a id="feateng"></a>

## Preprocessing Functions in BigQuery ML
You can use [TRANSFORM()](https://cloud.google.com/bigquery-ml/docs/bigqueryml-transform) with any standard SQL function or any ML [preprocessing function](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-preprocessing-functions). Some examples are shown below:

<a id="extractdates"></a>

#### EXTRACT: Extract dates and time

[EXTRACT from DATE documentation](https://cloud.google.com/bigquery/docs/reference/standard-sql/date_functions#extract)  
[EXTRACT from TIMESTAMP documentation](https://cloud.google.com/bigquery/docs/reference/standard-sql/timestamp_functions#extract)

In [ ]:
sql_datetime = """
    SELECT
        earliest_cr_line,
        EXTRACT( year FROM earliest_cr_line) AS year,
        EXTRACT( month FROM earliest_cr_line) AS month,
        EXTRACT( dayofweek FROM earliest_cr_line) AS dayofweek,
        EXTRACT( day FROM earliest_cr_line) AS day,
    FROM
        kaggle-hackathon.credit.train
    LIMIT 10
    """

bq_query(sql_datetime)

#### ML.QUANTILE_BUCKETIZE: Bucketize your numeric feature into bins
[ML.QUANTILE_BUCKETIZE Documentation](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-preprocessing-functions#quantile_bucketize)

In [ ]:
sql_bucketize = """
SELECT 
    loan_amnt,
    ML.QUANTILE_BUCKETIZE(
      loan_amnt,
      100 # number of buckets
    ) OVER() AS bucket_funded_amnt,
FROM
    kaggle-hackathon.credit.train
LIMIT 10
"""

bq_query(sql_bucketize)

<a id="polynomialexpand"></a>

#### ML.POLYNOMIAL_EXPAND: Combine numeric features

[ML.POLYNOMIAL_EXPAND Documentation](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-preprocessing-functions#mlpolynomial_expand)

In [ ]:
sql_polynomialexpand = """
SELECT
    loan_amnt,
    int_rate,
    ML.POLYNOMIAL_EXPAND(
      STRUCT(loan_amnt, int_rate), 
      2
    ) AS x
FROM
    kaggle-hackathon.credit.train
LIMIT 3
"""

bq_query(sql_polynomialexpand)

<a id="featurecross"></a>

#### ML.FEATURE_CROSS: Combine categorical features

[ML.FEATURE_CROSS Documentation](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-preprocessing-functions#mlfeature_cross)

In [ ]:
sql_featurecross = """
SELECT
    grade,
    sub_grade,
    ML.FEATURE_CROSS(
      STRUCT(grade, sub_grade), 
      2
    ) AS x
FROM
    kaggle-hackathon.credit.train
LIMIT 3
"""

bq_query(sql_featurecross)

<a id="featselection"></a>
<hr>

## Feature Selection

<a id="featimportance"></a>

### ML.FEATURE_IMPORTANCE for Boosted Tree models

_Note: This function is only available for Boosted Tree models (`boosted_tree_classifier` or `boosted_tree_regressor`)._

The ML.FEATURE_IMPORTANCE function allows you to see feature importance score, which indicates how useful or valuable each feature was in the construction of the Boosted Tree model during training. [ML.FEATURE_IMPORTANCE Documentation](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-importance#mlfeature_importance_function)

In [ ]:
MODEL_NAME = "MY_BOOSTED_TREE_MODEL_NAME"

sql_featureimportance = f"""
SELECT
    *
FROM
    ML.FEATURE_IMPORTANCE(MODEL {DATASET_ID}.{MODEL_NAME})
"""

bq_query(sql_featureimportance)